<a href="https://colab.research.google.com/github/ajaysuseel/MiniProject_AD/blob/main/finetuning_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from transformers import BlipProcessor, BlipForConditionalGeneration
from pytorch_lightning.callbacks import ModelCheckpoint
import json
import requests
from PIL import Image
from torchvision import transforms

In [2]:
# Configurable dataset path from GitHub
GITHUB_REPO = "https://raw.githubusercontent.com/ajaysuseel/MiniProject_AD/main/augmented/"
JSON_FILE = "captions.json"
IMAGES_FOLDER = "images/"

In [3]:
# Dataset Class
class CaptionDataset(Dataset):
    def __init__(self, processor):
        json_url = GITHUB_REPO + JSON_FILE
        try:
            response = requests.get(json_url)
            response.raise_for_status()
            self.data = response.json()
            print(f"Loaded {len(self.data)} image-caption pairs.")
        except requests.exceptions.RequestException as e:
            print(f"Error loading dataset: {e}")
            self.data = []

        self.processor = processor
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image_url = GITHUB_REPO + IMAGES_FOLDER + item["filename"]

        try:
            image = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")
        except Exception as e:
            print(f"Error loading image {item['filename']}: {e}")
            return None

        pixel_values = self.transform(image)
        encoding = self.processor(
            text=item["description"],
            padding="max_length",
            return_tensors="pt"
        )

        return {
            "pixel_values": pixel_values,
            "input_ids": encoding.input_ids.squeeze(0),
            "attention_mask": encoding.attention_mask.squeeze(0),
            "labels": encoding.input_ids.squeeze(0)
        }


In [11]:
# BLIP Lightning Module
class BlipLightning(pl.LightningModule):
    def __init__(self, model_name="Salesforce/blip-image-captioning-base", learning_rate=5e-5):
        super().__init__()
        self.model = BlipForConditionalGeneration.from_pretrained(model_name)
        self.processor = BlipProcessor.from_pretrained(model_name, use_fast=True)
        self.learning_rate = learning_rate

    def forward(self, pixel_values, input_ids, attention_mask, labels):
        return self.model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

    def training_step(self, batch, batch_idx):
        outputs = self(
            pixel_values=batch["pixel_values"],
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )
        loss = outputs.loss
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

In [13]:
# DataLoader Wrapper
def create_dataloader(batch_size=4):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base",use_fast=True)
    dataset = CaptionDataset(processor)

    def collate_fn(batch):
        batch = [b for b in batch if b is not None]
        if len(batch) == 0:
            return None
        keys = batch[0].keys()
        return {key: torch.stack([b[key] for b in batch]) for key in keys}

    return DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)


In [12]:
# Training Function
def train_model(num_epochs=3):
    dataloader = create_dataloader()

    checkpoint_callback = ModelCheckpoint(
        monitor="train_loss", mode="min", save_top_k=1, dirpath="checkpoints/", filename="blip_best"
    )

    trainer = pl.Trainer(max_epochs=num_epochs, callbacks=[checkpoint_callback], accelerator="auto")
    model = BlipLightning()

    model.train()#train mode
    trainer.fit(model, dataloader)

    print("✅ Fine-tuning complete. Best model saved!")

In [14]:
# Run Training
if __name__ == "__main__":
    train_model(num_epochs=7)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded 1270 image-caption pairs.


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                         | Params | Mode 
---------------------------------------------------------------
0 | model | BlipForConditionalGeneration | 247 M  | train
---------------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
989.656   Total estimated model params size (MB)
491       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=7` reached.


✅ Fine-tuning complete. Best model saved!


In [18]:
from transformers import BlipProcessor

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
processor.save_pretrained("checkpoints/blip_processor")

[]

#Gdrive

In [19]:
import os
from google.colab import drive

# 🚀 Mount Google Drive
drive.mount('/content/drive')

# 🔄 Define source & destination paths
MODELS_SRC = "/content/checkpoints"
GDRIVE_DEST = "/content/drive/MyDrive/lightning_models/models"

# ✅ Copy models folder to Google Drive
if os.path.exists(MODELS_SRC):
    !cp -r {MODELS_SRC} {GDRIVE_DEST}
    print(f"✅ Checkpont folder successfully copied to Google Drive: {GDRIVE_DEST}")
else:
    print("⚠️ No 'checkpoint' folder found in /content/. Please check your path.")

Mounted at /content/drive
✅ Checkpont folder successfully copied to Google Drive: /content/drive/MyDrive/lightning_models/models


In [20]:
# 🔄 Define source & destination paths
MODELS_SRC = "/content/lightning_logs"
GDRIVE_DEST = "/content/drive/MyDrive/lightning_models/logs"

# ✅ Copy models folder to Google Drive
if os.path.exists(MODELS_SRC):
    !cp -r {MODELS_SRC} {GDRIVE_DEST}
    print(f"✅ Logs folder successfully copied to Google Drive: {GDRIVE_DEST}")
else:
    print("⚠️ No 'logs' folder found in /content/. Please check your path.")

✅ Logs folder successfully copied to Google Drive: /content/drive/MyDrive/lightning_models/logs


In [21]:
drive.flush_and_unmount()

#Evaluation